In [1]:
import sapien.core as sapien
import numpy as np
from sapien.core import Pose
import time
import matplotlib.pyplot as plt
from transforms3d.taitbryan import euler2quat
from transforms3d.euler import quat2euler

%matplotlib notebook

Using default glsl path /home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/sapien/glsl_shader/130


# Construct Scene

In [2]:
sim = sapien.Engine()
renderer = sapien.OptifuserRenderer()
sim.set_renderer(renderer)
render_controller = sapien.OptifuserController(renderer)


def create_scene(timestep, visual):
    s = sim.create_scene()
    s.add_ground(-2)
    s.set_timestep(timestep)

    loader = s.create_urdf_loader()
    loader.fix_root_link = 0
    if visual:
        loader.collision_is_visual = True
        s.set_ambient_light([0.5, 0.5, 0.5])
        s.set_shadow_light([0, 1, -1], [0.5, 0.5, 0.5])
    robot = loader.load("humanoid.urdf")
    
    return s, robot

s0, robot0 = create_scene(1/30, True)
# s1, robot1 = create_scene(1/30, False)
# s2, robot2 = create_scene(1/30, False)
    
render_controller.set_camera_position(-5, 0, 0)
render_controller.set_current_scene(s0)

# Wrap IO

## State
    - COM pos (x y)
        - 13 * 2
    - foot pos (x y z)
        - additional 1
    - torso pos (x y z)
        - additional 1
    - COM velo?
        - additional 13 * 3
## Action
    - 21 params

In [3]:
target_body_parts=['torso', 'left_foot', 'right_foot']

def get_links(robot):
    '''
        Return all the link that have significant mass for the robot
        
        target_body_parts : [str body parts]
    '''
    links = {}
    
    for l in robot.get_links():
        name = l.get_name()
        mass = l.get_mass()
        if mass > 1:
            links[name] = l
        
    return links

def get_target_link(links, target_body_names):
    target_body_parts = {}
    
    for name, l in links.items():
        if name in target_body_names:
            target_body_parts[name] = l
    
    return target_body_parts

def report_link(l):
    '''
        Report name, mass, velocity, and position of the link.
    '''
    name = l.get_name()
    mass = l.get_mass()
    velo = l.get_velocity()
    pos = l.get_pose().p
    print(f"{name}\n\tmass: {mass:.6f}\n\tvelo: {velo}\n\tpos:  {pos}\n")

def report_all_links(ls):
    '''
        Repot info for all links.
    '''
    for _, l in ls.items():
        report_link(l)
        
def get_mass_dic(links):
    '''
        Input:
            links: the links dict
        
        Output:
            array of masses for each link
    '''
    mass = []
    for name, l in ls.items():
        mass.append(l.get_mass())
    return np.array(mass)

def get_pos(links):
    '''
        Input:
            links: the links dict (n_l # links)
            
        Output:
            array of shape (n_l, 3)
    '''
    pos = []
    for name, l in links.items():
        pos.append(l.get_pose().p)
    return np.array(pos)

def get_state(links):
    # Only using position now, might use velocity for COM VELO constraint
    return get_pos(links).flatten()

In [4]:
# the derivaties
def num_fx(u, scene, robot, links=None, eps=0.00001):
    '''
        Only doing pos now, return (n_x, n_x)
    '''
    if links == None:
        links = get_links(robot)
    
    ini_state = robot.pack()
    
    res = []
    
    for _, l in links.items():
        for i in range(3):
            robot.unpack(ini_state)
            
            # modify 
            orig_p = l.get_pose()
            pos = orig_p.p
            pos[i] += eps
            orig_p.set_p(pos)
            
            # simulate
            robot.set_qf(u)
            scene.step()
            
            res.append(get_state(links) / eps)
    return np.array(res).T
    
def num_fu(u, scene, robot, links=None, eps=0.00001):
    '''
        Only doing pos now, return (n_x, n_u)
    '''
    if links == None:
        links = get_links(robot)
    
    ini_state = robot.pack()
    
    res = []
    
    for i in range(robot.dof):
        robot.unpack(ini_state)
        
        # modify
        new_u = u.copy()
        new_u[i] += eps
        
        # simulate
        robot.set_qf(new_u)
        scene.step()
        
        res.append(get_state(links) / eps)
    return np.array(res).T
        

In [ ]:
# cnt = 0
# prev_clk = time.clock()
# # render_controller.show_window()
# robot.set_root_pose(Pose([0,0,5]))
# fps = []

# fig = plt.figure()
# ax = fig.add_subplot(111)

# fig.show()
# fig.canvas.draw()

# while True:
#     ctrl = np.random.rand(robot.dof) * 10
#     robot.set_qf(ctrl)
    
#     cnt += 1
#     if time.clock() - prev_clk >= 1:
#         prev_clk = time.clock()
#         fps.append(cnt)
#         cnt = 0
        
#         ax.clear()
#         ax.plot(fps)
#         fig.canvas.draw()
#     s0.step()
# #     s0.update_render()
    
# #     render_controller.focus(robot.get_links()[0])
# #     render_controller.render()